# CrewAI: Zero to Hero Guide

## Orchestrating Role-Playing Collaborative Agent Teams

**Objective:** This comprehensive notebook takes you from beginner to advanced CrewAI user. You'll learn how to create sophisticated multi-agent "crews" that work together through role-playing to accomplish complex goals.

**Target Audience:** Software engineers from complete beginners to experts looking to master CrewAI.

---

## Table of Contents
1. [Introduction & Core Philosophy](#1-introduction--core-philosophy)
2. [Prerequisites & Setup](#2-prerequisites--setup)
3. [Core Concepts Deep Dive](#3-core-concepts-deep-dive)
4. [Your First Crew: Basic Setup](#4-your-first-crew-basic-setup)
5. [Agent Configuration in Detail](#5-agent-configuration-in-detail)
6. [Task Design & Dependencies](#6-task-design--dependencies)
7. [Process Types: Sequential vs Hierarchical](#7-process-types-sequential-vs-hierarchical)
8. [Custom Tools & Capabilities](#8-custom-tools--capabilities)
9. [Memory & Context Management](#9-memory--context-management)
10. [Advanced Crew Patterns](#10-advanced-crew-patterns)
11. [Best Practices & Common Pitfalls](#11-best-practices--common-pitfalls)
12. [Conclusion & Next Steps](#12-conclusion--next-steps)

---

## 1. Introduction & Core Philosophy

### What is CrewAI?

**CrewAI** is a framework for orchestrating role-playing, autonomous AI agents. Think of it as assembling a team of specialists who each have their own expertise, personality, and responsibilities.

### Core Philosophy

| Principle | Description |
|-----------|-------------|
| **Role-Playing** | Agents have roles, goals, and backstories that guide their behavior |
| **Task-Oriented** | Work is organized into discrete, assignable tasks |
| **Process-Driven** | Execution follows defined processes (sequential or hierarchical) |
| **Tool-Equipped** | Agents can use tools to interact with external systems |

### When to Use CrewAI?

✅ **Good for:**
- Complex workflows requiring specialized expertise
- Content creation pipelines (research → write → edit)
- Analysis and report generation
- Projects benefiting from clear role separation

❌ **Consider alternatives when:**
- You need simple single-agent tasks (use LangChain)
- You need complex state management (use LangGraph)
- You need conversational multi-agent systems (use AutoGen)

### CrewAI vs Other Frameworks

```
┌────────────────┬──────────────────────────────────────────┐
│   Framework    │   Best For                               │
├────────────────┼──────────────────────────────────────────┤
│   CrewAI       │   Role-playing crews, structured tasks   │
│   AutoGen      │   Conversational problem-solving         │
│   LangGraph    │   Complex state machines, control flow   │
│   LangChain    │   Single-agent tools and chains          │
└────────────────┴──────────────────────────────────────────┘
```

---

## 2. Prerequisites & Setup

### Requirements

Before starting, ensure you have:

- **Python 3.10+** (CrewAI requires modern Python)
- **OpenAI API Key** (or compatible API)
- **Tavily API Key** (optional, for web search tools)

### Installation

```bash
# Install CrewAI with tools support
pip install crewai crewai-tools

# For additional integrations
pip install tavily-python  # Web search
```

In [ ]:
# Install dependencies (uncomment to run)
# !pip install crewai crewai-tools python-dotenv tavily-python

In [ ]:
import os
import warnings
from dotenv import load_dotenv

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Verify API keys
openai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

print("🔑 API KEY STATUS")
print("-" * 40)
print(f"OpenAI API Key: {'✅ Found' if openai_key else '❌ Missing'}")
print(f"Tavily API Key: {'✅ Found' if tavily_key else '⚠️ Optional - Missing'}")

if not openai_key:
    print("\n⚠️ Please create a .env file with: OPENAI_API_KEY=your_key_here")

---

## 3. Core Concepts Deep Dive

CrewAI has four fundamental building blocks:

### The Four Pillars

```
┌─────────────────────────────────────────────────────────────────────┐
│                         CrewAI Architecture                         │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   ┌─────────┐    ┌─────────┐    ┌─────────┐    ┌─────────┐        │
│   │  Agent  │    │  Task   │    │  Crew   │    │ Process │        │
│   ├─────────┤    ├─────────┤    ├─────────┤    ├─────────┤        │
│   │ • Role  │    │ • Desc  │    │ • Agents│    │Sequential│        │
│   │ • Goal  │    │ • Agent │    │ • Tasks │    │    or   │        │
│   │ • Back- │    │ • Output│    │ • Process│   │Hierarchi-│        │
│   │   story │    │         │    │         │    │   cal   │        │
│   │ • Tools │    │         │    │         │    │         │        │
│   └─────────┘    └─────────┘    └─────────┘    └─────────┘        │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 1. Agent
An autonomous unit with:
- **Role**: What the agent does (e.g., "Senior Researcher")
- **Goal**: What the agent aims to achieve
- **Backstory**: Context that shapes behavior
- **Tools**: External capabilities the agent can use

### 2. Task
A specific unit of work with:
- **Description**: What needs to be done
- **Agent**: Who is responsible
- **Expected Output**: What the result should look like
- **Context**: Tasks that provide input

### 3. Crew
The orchestrating container:
- Collection of agents
- Collection of tasks
- Process configuration
- Memory settings

### 4. Process
How tasks are executed:
- **Sequential**: Tasks run in order
- **Hierarchical**: A manager delegates work

---

## 4. Your First Crew: Basic Setup

Let's create a simple two-agent crew for travel planning.

In [ ]:
from crewai import Agent, Task, Crew, Process

print("✅ CrewAI imported successfully!")
print(f"   Version: Check with 'pip show crewai'")

In [ ]:
# Step 1: Define the Agents

# Travel Expert Agent
travel_agent = Agent(
    role="Expert Travel Agent",
    goal="Create amazing travel itineraries that provide unforgettable experiences",
    backstory="""You are a seasoned travel expert with 20 years of experience 
    planning trips worldwide. You have insider knowledge of the best destinations, 
    hidden gems, and optimal timing for visits. You pride yourself on creating 
    personalized itineraries that balance popular attractions with unique experiences.""",
    verbose=True,  # Show agent's thought process
    allow_delegation=False,  # Handle tasks directly
)

# Food Critic Agent
food_critic = Agent(
    role="Local Food Critic",
    goal="Recommend authentic and memorable dining experiences",
    backstory="""You are a renowned food critic who has traveled extensively 
    and written for major culinary publications. You specialize in finding 
    authentic local restaurants that tourists often miss. You understand 
    different cuisines deeply and can match restaurants to travelers' preferences.""",
    verbose=True,
    allow_delegation=False,
)

print("✅ Agents created!")
print(f"   1. {travel_agent.role}")
print(f"   2. {food_critic.role}")

In [ ]:
# Step 2: Define the Tasks

# Task 1: Create itinerary
itinerary_task = Task(
    description="""Create a 3-day travel itinerary for a trip to Tokyo, Japan.
    Focus on cultural sites and activities. Include:
    - Morning, afternoon, and evening activities for each day
    - Brief descriptions of why each activity is recommended
    - Practical tips for each location""",
    expected_output="A detailed 3-day itinerary with activities organized by time of day",
    agent=travel_agent,
)

# Task 2: Add restaurant recommendations
dining_task = Task(
    description="""Based on the planned itinerary, recommend one authentic 
    restaurant for each day of the trip. Consider:
    - Location proximity to day's activities
    - Authentic local cuisine
    - Range of price points
    - Special dishes to try at each restaurant""",
    expected_output="Restaurant recommendations for each day with specific dish suggestions",
    agent=food_critic,
    context=[itinerary_task],  # This task depends on the itinerary
)

print("✅ Tasks created!")
print(f"   1. Itinerary Task → {travel_agent.role}")
print(f"   2. Dining Task → {food_critic.role} (depends on Task 1)")

In [ ]:
# Step 3: Assemble the Crew

travel_crew = Crew(
    agents=[travel_agent, food_critic],
    tasks=[itinerary_task, dining_task],
    process=Process.sequential,  # Tasks run in order
    verbose=True,
)

print("✅ Crew assembled!")
print(f"   Agents: {len(travel_crew.agents)}")
print(f"   Tasks: {len(travel_crew.tasks)}")
print(f"   Process: Sequential")

In [ ]:
# Step 4: Kick off the Crew!

print("\n" + "="*60)
print("🚀 STARTING TRAVEL PLANNING CREW")
print("="*60 + "\n")

result = travel_crew.kickoff()

print("\n" + "="*60)
print("✅ CREW EXECUTION COMPLETE")
print("="*60)

In [ ]:
# Step 5: View the Results

print("\n📋 FINAL OUTPUT")
print("-" * 60)
print(result)

### 🎯 Key Takeaways from Basic Example

1. **Agents need clear roles, goals, and backstories** - these guide their behavior
2. **Tasks define what needs to be done** - with clear expected outputs
3. **Context creates dependencies** - tasks can build on each other's results
4. **Process determines execution order** - sequential runs tasks in order

---

## 5. Agent Configuration in Detail

Let's explore all the configuration options for agents.

In [ ]:
from crewai import Agent, LLM

# Full Agent Configuration Example
fully_configured_agent = Agent(
    # Core Identity
    role="Senior Software Architect",
    goal="Design scalable, maintainable software systems",
    backstory="""You are a senior architect with 15 years of experience 
    designing systems for Fortune 500 companies. You've seen patterns 
    succeed and fail, and you always consider long-term maintainability.""",
    
    # Behavior Settings
    verbose=True,            # Show thought process
    allow_delegation=True,   # Can delegate to other agents
    
    # LLM Configuration (optional - uses OpenAI by default)
    # llm=LLM(model="gpt-4o", temperature=0.7),
    
    # Memory & Context
    memory=True,             # Enable memory
    max_iter=15,             # Max iterations for a task
    max_rpm=10,              # Rate limit (requests per minute)
    
    # Tools (we'll cover this later)
    tools=[],
)

print("✅ Fully configured agent created!")
print(f"   Role: {fully_configured_agent.role}")
print(f"   Delegation: {fully_configured_agent.allow_delegation}")
print(f"   Memory: Enabled")

In [ ]:
# Different Agent Personalities for the Same Role

# Creative approach
creative_writer = Agent(
    role="Content Writer",
    goal="Create engaging, innovative content that captivates readers",
    backstory="""You're a creative writer known for thinking outside the box.
    Your content is memorable, uses unexpected metaphors, and challenges
    conventional thinking. You believe boring content is worse than no content.""",
    verbose=True,
)

# Technical approach
technical_writer = Agent(
    role="Content Writer",
    goal="Create clear, accurate, and well-structured content",
    backstory="""You're a technical writer with a background in engineering.
    You value precision, clarity, and logical structure above all else.
    Your content is always factual, well-researched, and easy to follow.""",
    verbose=True,
)

print("✅ Two different writer personalities created!")
print("\nCreative Writer Goal:", creative_writer.goal)
print("Technical Writer Goal:", technical_writer.goal)

---

## 6. Task Design & Dependencies

Tasks are the work units in CrewAI. Proper task design is crucial for good results.

In [ ]:
from crewai import Task

# Task Configuration Options

# Example: Research Task
research_task = Task(
    description="""Research the current state of AI in healthcare.
    
    Focus areas:
    1. Current applications in diagnostics
    2. Recent breakthroughs (last 2 years)
    3. Regulatory challenges
    4. Future predictions from experts
    
    Provide specific examples and cite sources where possible.""",
    
    expected_output="""A comprehensive research summary with:
    - Executive summary (2-3 sentences)
    - Key findings by focus area
    - Notable statistics or data points
    - List of sources consulted""",
    
    agent=creative_writer,  # Using the writer we created above
    
    # Optional: Output file
    # output_file="research_output.md",
)

print("✅ Research task configured!")

In [ ]:
# Task Dependencies with Context

# Task A: No dependencies
task_a = Task(
    description="Research market trends for electric vehicles",
    expected_output="Market analysis report",
    agent=creative_writer,
)

# Task B: Depends on Task A
task_b = Task(
    description="Based on the market research, identify the top 3 opportunities",
    expected_output="List of 3 opportunities with justification",
    agent=technical_writer,
    context=[task_a],  # Will receive task_a's output as context
)

# Task C: Depends on both A and B
task_c = Task(
    description="Create a strategic recommendation based on research and opportunities",
    expected_output="Strategic recommendation document",
    agent=creative_writer,
    context=[task_a, task_b],  # Will receive both outputs as context
)

print("✅ Task dependency chain created!")
print("\nExecution flow:")
print("   Task A (Research) → Task B (Opportunities) → Task C (Strategy)")

---

## 7. Process Types: Sequential vs Hierarchical

CrewAI supports two main process types for orchestrating tasks.

In [ ]:
from crewai import Crew, Process

# SEQUENTIAL PROCESS
# Tasks run in order, one after another

sequential_crew = Crew(
    agents=[creative_writer, technical_writer],
    tasks=[task_a, task_b, task_c],
    process=Process.sequential,  # ← Sequential execution
    verbose=True,
)

print("SEQUENTIAL PROCESS")
print("-" * 40)
print("Tasks execute in order: A → B → C")
print("Each task completes before the next starts")
print("Good for: Linear workflows, clear dependencies")

In [ ]:
# HIERARCHICAL PROCESS
# A manager agent coordinates and delegates tasks

# Create a manager agent
manager_agent = Agent(
    role="Project Manager",
    goal="Coordinate the team to deliver high-quality work efficiently",
    backstory="""You are an experienced project manager who excels at 
    breaking down complex projects, assigning the right people to tasks, 
    and ensuring quality delivery on time.""",
    verbose=True,
    allow_delegation=True,
)

hierarchical_crew = Crew(
    agents=[manager_agent, creative_writer, technical_writer],
    tasks=[task_a, task_b, task_c],
    process=Process.hierarchical,  # ← Hierarchical execution
    manager_agent=manager_agent,    # ← Specify the manager
    verbose=True,
)

print("\nHIERARCHICAL PROCESS")
print("-" * 40)
print("Manager coordinates and delegates tasks")
print("Manager decides optimal task assignment")
print("Good for: Complex projects, dynamic task allocation")

In [ ]:
# Comparison Diagram
print("""
┌─────────────────────────────────────────────────────────────────┐
│                     PROCESS COMPARISON                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   SEQUENTIAL                    HIERARCHICAL                    │
│                                                                 │
│   ┌─────────┐                  ┌─────────────┐                 │
│   │ Task A  │                  │   Manager   │                 │
│   └────┬────┘                  └──────┬──────┘                 │
│        ▼                         ┌────┴────┐                    │
│   ┌─────────┐                   ▼         ▼                    │
│   │ Task B  │              ┌─────────┐ ┌─────────┐             │
│   └────┬────┘              │ Agent A │ │ Agent B │             │
│        ▼                   └────┬────┘ └────┬────┘             │
│   ┌─────────┐                   ▼           ▼                  │
│   │ Task C  │              ┌─────────┐ ┌─────────┐             │
│   └─────────┘              │ Task X  │ │ Task Y  │             │
│                            └─────────┘ └─────────┘             │
│                                                                 │
│   Pros:                        Pros:                           │
│   • Predictable               • Dynamic allocation             │
│   • Easy to debug             • Better resource use            │
│   • Clear flow                • Adapts to complexity           │
│                                                                 │
│   Cons:                        Cons:                           │
│   • Rigid                     • More LLM calls                 │
│   • No parallelism            • Harder to debug                │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
""")

---

## 8. Custom Tools & Capabilities

Tools extend what agents can do beyond text generation.

In [ ]:
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

# Method 1: Create a custom tool using BaseTool

class CalculatorInput(BaseModel):
    """Input schema for calculator."""
    expression: str = Field(..., description="Mathematical expression to evaluate")

class CalculatorTool(BaseTool):
    name: str = "Calculator"
    description: str = "Evaluates mathematical expressions. Use for any calculations."
    args_schema: Type[BaseModel] = CalculatorInput
    
    def _run(self, expression: str) -> str:
        """Execute the calculation."""
        try:
            # Note: eval is used for demo - use a safe parser in production!
            result = eval(expression)
            return f"Result: {result}"
        except Exception as e:
            return f"Error: {str(e)}"

calculator = CalculatorTool()
print("✅ Custom Calculator Tool created!")
print(f"   Name: {calculator.name}")
print(f"   Description: {calculator.description}")

In [ ]:
# Method 2: Use the @tool decorator (simpler approach)

from crewai.tools import tool

@tool("Weather Lookup")
def weather_lookup(city: str) -> str:
    """Get the current weather for a city. Use when you need weather information."""
    # Simulated weather data
    weather_data = {
        "tokyo": "☀️ Sunny, 72°F (22°C), Humidity: 45%",
        "new york": "🌤️ Partly Cloudy, 65°F (18°C), Humidity: 60%",
        "london": "🌧️ Rainy, 55°F (13°C), Humidity: 80%",
        "paris": "⛅ Cloudy, 60°F (16°C), Humidity: 65%",
    }
    city_lower = city.lower()
    return weather_data.get(city_lower, f"Weather data not available for {city}")

print("✅ Weather Lookup Tool created!")
print(f"   Test: {weather_lookup('Tokyo')}")

In [ ]:
# Method 3: Use built-in CrewAI tools

try:
    from crewai_tools import (
        SerperDevTool,      # Google search
        WebsiteSearchTool,  # Search specific websites
        FileReadTool,       # Read files
        DirectoryReadTool,  # Read directories
    )
    print("✅ CrewAI built-in tools available!")
except ImportError:
    print("⚠️ crewai-tools not installed. Run: pip install crewai-tools")

In [ ]:
# Create an agent with tools

research_agent = Agent(
    role="Research Analyst",
    goal="Gather accurate information using available tools",
    backstory="You are a meticulous researcher who verifies information from multiple sources.",
    tools=[calculator, weather_lookup],  # Assign tools to agent
    verbose=True,
)

print("✅ Agent with tools created!")
print(f"   Tools: Calculator, Weather Lookup")

---

## 9. Memory & Context Management

CrewAI supports different types of memory to help agents maintain context.

In [ ]:
# Memory Types in CrewAI
print("""
┌─────────────────────────────────────────────────────────────────┐
│                     MEMORY TYPES IN CREWAI                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   SHORT-TERM MEMORY                                            │
│   └── Context within current execution                         │
│   └── Automatically managed                                    │
│   └── Cleared after crew completes                            │
│                                                                 │
│   LONG-TERM MEMORY                                             │
│   └── Persists across executions                              │
│   └── Stores insights and learnings                           │
│   └── Requires explicit configuration                         │
│                                                                 │
│   ENTITY MEMORY                                                │
│   └── Remembers information about entities                    │
│   └── People, companies, concepts                             │
│   └── Useful for relationship tracking                        │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
""")

In [ ]:
# Crew with Memory Enabled

memory_agent = Agent(
    role="Customer Support Agent",
    goal="Provide helpful support while remembering customer context",
    backstory="You're a friendly support agent who remembers past interactions.",
    memory=True,  # Enable agent memory
    verbose=True,
)

support_task = Task(
    description="Help the customer with their inquiry about product returns.",
    expected_output="A helpful response addressing the customer's concern.",
    agent=memory_agent,
)

memory_crew = Crew(
    agents=[memory_agent],
    tasks=[support_task],
    process=Process.sequential,
    memory=True,  # Enable crew-level memory
    verbose=True,
)

print("✅ Memory-enabled crew created!")
print("   Agent memory: Enabled")
print("   Crew memory: Enabled")

---

## 10. Advanced Crew Patterns

Let's explore some advanced patterns for building effective crews.

In [ ]:
# Pattern 1: Content Creation Pipeline
# Research → Write → Edit → Review

# Define specialized agents
researcher = Agent(
    role="Senior Researcher",
    goal="Find accurate and relevant information on any topic",
    backstory="PhD researcher with expertise in finding and synthesizing information.",
    verbose=True,
)

writer = Agent(
    role="Content Writer",
    goal="Create engaging, well-structured content",
    backstory="Award-winning writer known for making complex topics accessible.",
    verbose=True,
)

editor = Agent(
    role="Editor",
    goal="Ensure content is polished, error-free, and impactful",
    backstory="Veteran editor who has worked for major publications.",
    verbose=True,
)

# Define pipeline tasks
research_task = Task(
    description="Research the topic: 'The Impact of AI on Modern Education'",
    expected_output="Comprehensive research notes with key facts and statistics",
    agent=researcher,
)

writing_task = Task(
    description="Write a 500-word article based on the research",
    expected_output="Well-structured article with introduction, body, and conclusion",
    agent=writer,
    context=[research_task],
)

editing_task = Task(
    description="Edit and polish the article for publication",
    expected_output="Publication-ready article with all errors corrected",
    agent=editor,
    context=[writing_task],
)

content_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    process=Process.sequential,
    verbose=True,
)

print("✅ Content Creation Pipeline created!")
print("\nPipeline: Research → Write → Edit")

In [ ]:
# Pattern 2: Analysis Team with Multiple Perspectives

optimist = Agent(
    role="Optimistic Analyst",
    goal="Identify opportunities and positive outcomes",
    backstory="You always see the glass half full and find potential in every situation.",
    verbose=True,
)

pessimist = Agent(
    role="Risk Analyst",
    goal="Identify risks and potential problems",
    backstory="You're the voice of caution who ensures all risks are considered.",
    verbose=True,
)

synthesizer = Agent(
    role="Strategic Synthesizer",
    goal="Combine different perspectives into balanced recommendations",
    backstory="You excel at finding truth in different viewpoints.",
    verbose=True,
)

# Tasks for balanced analysis
topic = "Launching a new AI product in the current market"

opportunity_task = Task(
    description=f"Analyze the opportunities in: {topic}",
    expected_output="List of opportunities with supporting rationale",
    agent=optimist,
)

risk_task = Task(
    description=f"Analyze the risks in: {topic}",
    expected_output="List of risks with mitigation suggestions",
    agent=pessimist,
)

synthesis_task = Task(
    description="Create a balanced strategic recommendation",
    expected_output="Strategic recommendation balancing opportunities and risks",
    agent=synthesizer,
    context=[opportunity_task, risk_task],
)

analysis_crew = Crew(
    agents=[optimist, pessimist, synthesizer],
    tasks=[opportunity_task, risk_task, synthesis_task],
    process=Process.sequential,
    verbose=True,
)

print("✅ Multi-Perspective Analysis Team created!")

In [ ]:
# Run the Analysis Team
print("\n" + "="*60)
print("🔍 RUNNING MULTI-PERSPECTIVE ANALYSIS")
print("="*60 + "\n")

analysis_result = analysis_crew.kickoff()

print("\n" + "="*60)
print("📊 ANALYSIS COMPLETE")
print("="*60)
print(analysis_result)

---

## 11. Best Practices & Common Pitfalls

### ✅ Best Practices

1. **Clear Role Definition**: Be specific about what each agent does
2. **Detailed Backstories**: Context improves agent behavior
3. **Explicit Expected Outputs**: Tell agents exactly what you want
4. **Task Dependencies**: Use context to chain task outputs
5. **Start Sequential**: Only use hierarchical when needed

### ❌ Common Pitfalls

1. **Vague Goals**: "Do good work" vs "Create a 500-word blog post"
2. **Missing Context**: Not connecting dependent tasks
3. **Too Many Agents**: Start with 2-3 agents and scale up
4. **No Verbose Mode**: Always debug with verbose=True first
5. **Ignoring Rate Limits**: Set max_rpm for API constraints

In [ ]:
# Production Configuration Template

from crewai import Agent, Task, Crew, Process

def create_production_crew():
    """Template for production-ready crew configuration."""
    
    # Define agents with comprehensive configuration
    agent_config = {
        "verbose": False,  # Disable in production
        "max_iter": 10,    # Prevent infinite loops
        "max_rpm": 10,     # Respect rate limits
        "memory": True,    # Enable memory
    }
    
    primary_agent = Agent(
        role="Primary Worker",
        goal="Complete assigned tasks efficiently and accurately",
        backstory="Experienced professional with deep domain expertise.",
        **agent_config,
    )
    
    reviewer_agent = Agent(
        role="Quality Reviewer",
        goal="Ensure all outputs meet quality standards",
        backstory="Detail-oriented reviewer who catches issues others miss.",
        **agent_config,
    )
    
    # Define tasks with explicit outputs
    main_task = Task(
        description="{task_description}",  # Parameterized
        expected_output="{expected_format}",
        agent=primary_agent,
    )
    
    review_task = Task(
        description="Review the output for quality and accuracy",
        expected_output="Reviewed and approved output with any corrections",
        agent=reviewer_agent,
        context=[main_task],
    )
    
    # Assemble crew
    crew = Crew(
        agents=[primary_agent, reviewer_agent],
        tasks=[main_task, review_task],
        process=Process.sequential,
        verbose=False,
        memory=True,
    )
    
    return crew

print("✅ Production crew template created!")

---

## 12. Conclusion & Next Steps

### What You've Learned

| Topic | Key Takeaway |
|-------|-------------|
| Core Concepts | Agents, Tasks, Crews, and Processes |
| Agent Design | Role, goal, and backstory shape behavior |
| Task Design | Clear descriptions and expected outputs |
| Process Types | Sequential vs Hierarchical execution |
| Tools | Extend agent capabilities |
| Memory | Maintain context across interactions |

### Next Steps

1. **Practice**: Build a crew for your specific use case
2. **Experiment**: Try different agent personalities and processes
3. **Compare**: See how LangGraph handles similar workflows
4. **Scale**: Add more agents and complex task dependencies

### Resources

- [CrewAI Documentation](https://docs.crewai.com/)
- [CrewAI GitHub](https://github.com/crewAIInc/crewAI)
- [CrewAI Tools](https://github.com/crewAIInc/crewAI-tools)

---

**Congratulations!** You've completed the CrewAI Zero to Hero guide! 🎉